#### To Start Off
- This project uses a dataset found on kaggle inorder to build an accurate based on 1400 recordings

In [ ]:
# Create new directory 
! mkdir dataset
# Download dataset from kaggle using my personal api which can be found on your kaggle account
! KAGGLE_USERNAME=ferasdahrooge1 KAGGLE_KEY=d65a33337f60ea5b0a5ee063dd09b34d kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio
# Unzip and save in newly created directory
! unzip ravdess-emotional-speech-audio.zip -d dataset

#### From the Dataset, Use the audio speech actors files to build a DataFrame

In [1]:
import os

dataset = "./dataset/audio_speech_actors_01-24"
actor_list = os.listdir(dataset)
path_list = []
gender_list = []
emotion_list = []

emotion_dictionary = {
  '01': 'neutral',
  '02': 'calm',
  '03': 'happy',
  '04': 'sad',
  '05': 'angry',
  '06': 'fearful',
  '07': 'disgust',
  '08': 'surprised'
}

for directory in actor_list:
    actor_files = os.listdir(os.path.join(dataset, directory))

    for audio_file in actor_files: 
        part = audio_file.split('.')[0]
        key = part.split('-')[2]
        if key in emotion_dictionary:
          gender_code = int(part.split('-')[6])
          path_list.append(f"{dataset}/{directory}/{audio_file}")
          gender_list.append('female' if gender_code % 2 == 0 else 'male')
          emotion_list.append(emotion_dictionary[key])

In [2]:
import pandas as pd

# Build DataFrame
dataframe = pd.concat([
  pd.DataFrame(path_list, columns=["path"]),
  pd.DataFrame(gender_list, columns=["gender"]),
  pd.DataFrame(emotion_list, columns=["emotion"])
] , axis=1)

In [3]:
dataframe

,path,gender,emotion
0,./dataset/audio_speech_actors_01-24/Actor_16/0...,female,angry
1,./dataset/audio_speech_actors_01-24/Actor_16/0...,female,fearful
2,./dataset/audio_speech_actors_01-24/Actor_16/0...,female,fearful
3,./dataset/audio_speech_actors_01-24/Actor_16/0...,female,angry
4,./dataset/audio_speech_actors_01-24/Actor_16/0...,female,disgust
...,...,...,...
1435,./dataset/audio_speech_actors_01-24/Actor_08/0...,female,happy
1436,./dataset/audio_speech_actors_01-24/Actor_08/0...,female,happy
1437,./dataset/audio_speech_actors_01-24/Actor_08/0...,female,calm
1438,./dataset/audio_speech_actors_01-24/Actor_08/0...,female,calm
